This notebooks shows how to pretrain any language model easily

And checkout this amazing dataset by [Manav](https://www.kaggle.com/mdhamani) <br>
https://www.kaggle.com/mdhamani/goodreads-books-100k

which contains 100k goodreads books dataset, that I am going to use in this notebook


1. Pretrain Roberta Model: this notebook
2. Finetune Roberta Model: https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune
3. Inference oNtebook: https://www.kaggle.com/maunish/clrp-pytorch-roberta-inference
4. Roberta + SVM: https://www.kaggle.com/maunish/clrp-roberta-svm


In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM, AutoTokenizer, LineByLineTextDataset,
                         DataCollatorForLanguageModeling,Trainer, TrainingArguments,)

In [2]:
train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

# goodreads = pd.read_csv('../input/goodreads-books-100k/GoodReads_100k_books.csv')
# goodreads['desc'] = goodreads['desc'].apply(lambda x: str(x))
# extra = pd.read_table('../input/clrp-external/weebit_reextracted.tsv')
# one_stop_english = pd.read_csv('../input/onestopenglishcorpus/all_data.csv')
# jigsaw = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')

text  = '.'.join(train_data.excerpt.tolist() + test_data.excerpt.tolist())

with open('text.txt','w') as f:
    f.write(text)

In [3]:
model_name = 'roberta-base'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention text file here
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir="./clrp_roberta_base_chk", #select model path
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=64,
    save_steps=130,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to = "none" 
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [5]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=134, training_loss=1.7393111043901586, metrics={'train_runtime': 159.2473, 'train_samples_per_second': 0.841, 'total_flos': 819004759308288.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 1606672384, 'init_mem_gpu_alloc_delta': 500087808, 'init_mem_cpu_peaked_delta': 380555264, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 152178688, 'train_mem_gpu_alloc_delta': 1502540800, 'train_mem_cpu_peaked_delta': 463134720, 'train_mem_gpu_peaked_delta': 13668094976})

In [6]:
trainer.save_model('./clrp_roberta_base')

In [7]:
model = AutoModel.from_pretrained('./clrp_roberta_base')

Some weights of RobertaModel were not initialized from the model checkpoint at ./clrp_roberta_base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
